In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
def rfm_variables(df):
  ## quantidade de vendas
  f_sales = len(df)
  ## quantidade de vendas com lucro
  f_profit = len(df[df['Profit'] > 0])
  ## total de vendas, lucro, quantidade e custo de entrega
  m_sales = round(df['Sales'].sum(),2)
  m_profit = round(df['Profit'].sum(),2)
  m_qty = df['Quantity'].sum()
  ## periodicidade
  df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates()
  df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
  df_sort['diff_int'] = df_sort['diff'].dt.days
  r_days = round(df_sort['diff_int'].mean(),2)
  return f_sales, f_profit, m_sales, m_profit, m_qty, r_days

In [3]:
def fit_data(data, variable):
  rfm = pd.DataFrame()
  variaveis = data[variable].unique()
  for variavel in variaveis:
    dados = data[data[variable] == variavel]
    f_vendas, f_lucro, m_vendas, m_lucro, m_qtde, r_dias = rfm_variables(dados)
    rfm = rfm.append(
        {
          'referencia': variavel, 
          'm_vendas': m_vendas, 
          'm_lucro': m_lucro,
          'm_qtde': m_qtde, 
          'r_dias': r_dias,
          'f_vendas': f_vendas, 
          'f_lucro': f_lucro           
        },
        ignore_index = True
    )
  return rfm

In [4]:
data = pd.read_feather('tavbase/Sample.feather')
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,...,OMonth,OPeriod,Order Date Month,sYear,sMonth,sPeriod,Ship Date Month,Delivery,Price,Benefit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,130.98,1
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,243.98,1
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,...,6,30,2016-06-01,2016,6,30,2016-06-01,4,7.31,1
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,191.52,0
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,11.18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,...,1,1,2014-01-01,2014,1,1,2014-01-01,2,8.42,1
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,45.98,1
9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,129.29,1
9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,7.40,1


In [5]:
variaveis = [
    'f_vendas',
    'f_lucro',
    'm_lucro',
    'm_qtde', 
    'm_vendas',
    'r_dias'
]

In [6]:
## Aula 09 - Clusterização ou Agrupamento
state_rfm = fit_data(data, 'State')
state_rfm = state_rfm.fillna(0)
state_rfm['cluster'] = KMeans(
    n_clusters=3,
    random_state=0
).fit(
    state_rfm[
        variaveis
    ]
).labels_
state_rfm

C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\2601909467.py:7: FutureWarning: The frame.append method is deprecated 

,referencia,m_vendas,m_lucro,m_qtde,r_dias,f_vendas,f_lucro,cluster
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0
1,California,457687.63,76381.39,7667,2.21,2001,1884,1
2,Florida,89473.71,-3399.30,1379,7.75,383,252,2
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0
4,Washington,138641.27,33402.65,1883,6.54,506,483,2
5,Texas,170188.05,-25729.36,3724,3.73,985,493,2
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0
7,Utah,11220.06,2546.53,219,53.16,53,53,0
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,2


In [7]:
cluster = pd.DataFrame()
for index, row in enumerate(KMeans(n_clusters=3, random_state=0
    ).fit(state_rfm[variaveis]).cluster_centers_):
    print(row)
    cluster = cluster.append(
        {
            'cluster'    : index,
            'clf_vendas' : row[0],
            'cls_lucro'  : row[1],
            'clm_lucro'  : row[2],
            'clm_qtde'   : row[3],
            'clm_vendas' : row[4],
            'clr_dias'   : row[5]
        },
        ignore_index = True
    )
cluster  

[   76.            66.69230769  3430.28717949   292.30769231
 18166.44717949    77.67564103]
[1.5645000e+03 1.4790000e+03 7.5209970e+04 5.9455000e+03 3.8428195e+05
 2.7800000e+00]
[4.87625000e+02 3.12375000e+02 2.74487500e+02 1.82275000e+03
 1.02518189e+05 7.89875000e+00]


C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\4237266428.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\4237266428.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_10868\4237266428.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(


,cluster,clf_vendas,cls_lucro,clm_lucro,clm_qtde,clm_vendas,clr_dias
0,0.0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
1,1.0,1564.500,1479.000000,75209.970000,5945.500000,384281.950000,2.780000
2,2.0,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750


In [8]:
state_rfm = state_rfm.merge(
    cluster,
    on='cluster',
    how='left'
)
state_rfm

,referencia,m_vendas,m_lucro,m_qtde,r_dias,f_vendas,f_lucro,cluster,clf_vendas,cls_lucro,clm_lucro,clm_qtde,clm_vendas,clr_dias
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
1,California,457687.63,76381.39,7667,2.21,2001,1884,1,1564.500,1479.000000,75209.970000,5945.500000,384281.950000,2.780000
2,Florida,89473.71,-3399.30,1379,7.75,383,252,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
4,Washington,138641.27,33402.65,1883,6.54,506,483,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750
5,Texas,170188.05,-25729.36,3724,3.73,985,493,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
7,Utah,11220.06,2546.53,219,53.16,53,53,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750


In [9]:
state_rfm.to_feather('tavbase/clusterizacao_estado.feather')